# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [1]:
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace

ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'CapstoneProjectHyperdrive'

experiment=Experiment(ws, experiment_name)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code F3NUX7H2W to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


In [2]:
# Import data from Kaggle

! pip install kaggle

import os
import zipfile
os.environ['KAGGLE_USERNAME'] = "chaamp"
os.environ['KAGGLE_KEY'] = "26be8486f6a035048aa3e8c49d9001cd"


! kaggle datasets download -d chaamp/language-recognition

path='language-recognition.zip'
tar='Users/odl_user_141832/'
with zipfile.ZipFile(path, 'r') as zip_ref:
    zip_ref.extractall('')

     |████████████████████████████████| 58 kB 3.4 MB/s eta 0:00:011
     |████████████████████████████████| 78 kB 4.5 MB/s  eta 0:00:01
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73053 sha256=ee7b3085fa828192dd31e4156a90ff06253b6b74701d72fa2628403b5ca1ba50
  Stored in directory: /home/azureuser/.cache/pip/wheels/77/47/e4/44a4ba1b7dfd53faaa35f59f1175e123b213ff401a8a56876b
  Created wheel for python-slugify: filename=python_slugify-4.0.1-py2.py3-none-any.whl size=6769 sha256=123d111bbc1def10c4d6847fa5120d85bce38441a0df912d0489f50a1a657645
  Stored in directory: /home/azureuser/.cache/pip/wheels/72/e6/db/122611605e60148f54ee2abaca98b2bbeafc6e22486a867bad
Successfully built kaggle python-slugify
 80%|██████████████████████████████▌       | 1.00M/1.24M [00:00<00:00, 2.49MB/s]
100%|██████████████████████████████████████| 1.24M/1.24M [00:00<00:00, 2.48MB/s]


In [3]:
! pip install openpyxl

     |████████████████████████████████| 243 kB 7.9 MB/s eta 0:00:01
  Created wheel for et-xmlfile: filename=et_xmlfile-1.0.1-py3-none-any.whl size=8917 sha256=b5b4a7bca1f99d3dca991cf510879a043c5521f677f130aeacb42e4522ff2b8f
  Stored in directory: /home/azureuser/.cache/pip/wheels/ef/92/e0/956c00decf3091294b8800d9104f6169a1adacfaafd122ed42
Successfully built et-xmlfile


In [4]:
# Import the data
from azureml.core.dataset import Dataset
import pandas as pd


key = 'LR data'
if key in ws.datasets.keys():
    dataset = ws.datasets[key]
    print("Dataset found")

else:
    df=pd.read_excel('trn.xlsx',engine='openpyxl',index_col=0)
    df.to_csv('trn.csv',index = False)
    datastore = ws.get_default_datastore()
    datastore.upload_files(files = ['./trn.csv'])
    train = Dataset.Tabular.from_delimited_files([(datastore,'trn.csv')])
    train = train.register(ws,key)

Uploading an estimated of 1 files
Uploading ./trn.csv
Uploaded ./trn.csv, 1 files out of an estimated total of 1
Uploaded 1 files


# Create Compute cluster

In [5]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException


cpu_cluster_name = "compute-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print('Creating a new compute cluster...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

# Can poll for a minimum number of nodes and for a specific timeout. 
# If no min node count is provided it uses the scale settings for the cluster.
compute_target.wait_for_completion(show_output=True)

Creating a new compute cluster...
Creating...
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [6]:
from azureml.core import ScriptRunConfig
from azureml.core import Environment
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice
import os
import shutil

In [7]:
ds_file = Dataset.File.from_files(path = [(datastore, ('trn.csv'))])

In [27]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor=0.15, evaluation_interval=1, delay_evaluation=10)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling( {
    'nlayers': choice(1,2,3,4),
    'nodes': choice(80,128,176,256),
    })

# create environment
curated_env_name = 'AzureML-TensorFlow-2.2-GPU'
tf_env = Environment.get(workspace=ws, name=curated_env_name)


# configure and submit your training run
run_config = ScriptRunConfig(source_directory='.',
                            script='train.py',
                            compute_target=compute_target,
                            arguments=['--data_path', ds_file.as_named_input('input').as_mount()],
                            environment=tf_env)

hyperdrive_run_config = HyperDriveConfig(run_config=run_config,
                                     hyperparameter_sampling=param_sampling, 
                                     policy=early_termination_policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=1,
                                     max_concurrent_runs=4)



In [28]:
#TODO: Submit your experiment
run = experiment.submit(config=hyperdrive_run_config)

In [29]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service